In [1]:
%%time

import sys
sys.path.append('../../')

import datetime as dt
import os

import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm

from common.captioning.caption import BlipCaption
from common.schemas.pyarrow_schema import schema
from common.storage.azure_file_storage import AzureFileStorageAdapter
from common.functions.functions import Functions

tqdm.pandas(desc="Progress")

file_system = AzureFileStorageAdapter('data').get_file_storage()

functions: Functions = Functions()

CPU times: total: 1.69 s
Wall time: 10 s


In [ ]:
%%time

# "memes","CityPorn", "EarthPorn", "spaceporn",	"itookapicture",	"trippinthroughtime",
subs = [
	"SFWRedheads",
	"sfwpetite",
	"SFWNextDoorGirls",
	"fatsquirrelhate",
	"realasians",
	"KoreanHotties",
	"prettyasiangirls",
	"AsianOfficeLady",
	"mildlypenis",
	"AsianInvasion",
	"sexygirls",
	"PrettyGirls",
	"gentlemanboners",
	"hotofficegirls",
	"tightdresses",
	"DLAH",
	"bathandbodyworks",
	"AesPleasingAsianGirls"
]

In [ ]:
%%time

sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": ["selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyAsianDiffusion","data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion","AesPleasingAsianGirls"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion","data": ["sexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls", "tightdresses", "DLAH"]},
	{"name": "CandleDiffusion", "data": ["bathandbodyworks"]}
]
sources_df = pd.DataFrame.from_records(sources)

In [ ]:
%%time

extant_data = pandas.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system,
								  schema=schema)
extant_data.set_index('id', inplace=True)
display(extant_data)

In [ ]:
%%time

curated_data = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
curated_data.set_index('id', inplace=True)

display("==== CURATED DATA ====")
display(curated_data.shape)
display(curated_data)

In [ ]:
%%time

extant_data.update(curated_data)
updated_extant_data = extant_data.reset_index()

display("==== UPDATED EXTANT DATA ====")
display(updated_extant_data.shape)
display(updated_extant_data)
updated_extant_data.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

In [ ]:
%%time

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA', user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

In [ ]:
for sub in tqdm(subs, total=len(subs), desc="Creating Temp Dir For Subs..."):
	temp_dir_path_ = f"temp/{sub}"
	if not os.path.exists(temp_dir_path_):
		os.makedirs(temp_dir_path_)
	os.makedirs(temp_dir_path_, exist_ok=True)

In [ ]:
with open('log.txt', 'a') as f:
	for time_filter in ['week', 'day']:
		for sub in tqdm(subs, desc=f"{time_filter}", total=len(subs)):
			temp_dir_path = f"temp/{sub}"
			try:
				subreddit_stream: ListingGenerator = reddit.subreddit(display_name=sub).top(limit=100, time_filter=time_filter)
				subreddit_stream = list(subreddit_stream)
				for submission in tqdm(subreddit_stream, total=len(subreddit_stream),
									   desc=f"Posts - {sub} - {time_filter}"):
					if submission is None:
						continue
					else:
						if submission.id in extant_data.index.values:
							continue
					try:
						author_name = 'Unknown'
						subreddit_name = sub
						try:
							author_name = submission.author.name
						except Exception as e:
							author_name = 'Unknown'
							pass
						p = {
							'id': submission.id,
							'subreddit': subreddit_name,
							'author': author_name,
							'title': submission.title,
							'caption': '',
							'hash': '',
							'permalink': submission.permalink,
							'original_url': submission.url,
							'image_name': '',
							'path': '',
							'thumbnail_path': '',
							'exists': False,
							'curated': False,
							'Tags': ['']
						}
						pd.DataFrame([p]).to_parquet(f"{temp_dir_path}/{submission.id}.parquet")
					except Exception as e:
						log = functions.write_log_message(submission.id, sub, "Error Writing Post", e)
						f.write(log)
						continue
			except Exception as e:
				log = functions.write_log_message(submission.id, sub, f"Error Getting Posts For SubReddit", e)
				f.write(log)
				continue

In [ ]:
%%time

data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pandas.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

temp_data = pandas.DataFrame(data)

for i, r in temp_data.iterrows():
	temp_data.loc[i, 'image_name'] = r.id + ".jpg"
	temp_data.loc[i, 'path'] = ""
	temp_data.loc[i, 'hash'] = ""
	temp_data.loc[i, 'caption'] = ""
	temp_data.loc[i, 'model'] = ""
	temp_data.loc[i, 'exists'] = False
	temp_data.loc[i, 'curated'] = False
	temp_data.loc[i, 'accept'] = False
	temp_data.loc[i, 'tags'] = ['']

temp_data.set_index('id', inplace=True, drop=False)

display("==== TEMP DATA ====")
display(temp_data.shape)
display(temp_data)

In [ ]:
%%time

filtered = temp_data[~temp_data['id'].isin(extant_data.index.values) & temp_data['original_url'].str.endswith('.jpg') & (~temp_data['id'].isin(curated_data.index.values))]
filtered.dropna(axis=1, how='all')
filtered.reset_index(drop=True, inplace=True)
filtered.set_index('id', inplace=True, drop=False)
display("==== FILTERED DATA ====")
display(filtered.shape)
display(filtered)

In [ ]:
%%time

filtered['model'] = filtered.progress_apply(lambda x: functions.add_source(x, sources), axis=1)

display("== Filtered Data With Model ==")
display(filtered.shape)
display(filtered)

In [ ]:
%%time

file_list = file_system.ls("data/image")

filtered['path'] = filtered.progress_apply(lambda x: functions.fetch_image(x, file_list, file_system), axis=1)

display("== Filtered Data With Path ==")
display(filtered.shape)
display(filtered)

In [ ]:
%%time

filtered['exists'] = filtered.progress_apply(lambda x: functions.set_exists(x), axis=1)

display("== Filtered Data With Exists ==")
display(filtered.shape)
display(filtered)

In [ ]:
%%time

filtered['hash'] = filtered.progress_apply(lambda x: functions.set_hash(x), axis=1)

display("== Filtered Data With Hash ==")
display(filtered.shape)
display(filtered)

In [ ]:
%%time
import torch
if torch.cuda.is_available():
	caption_0 = BlipCaption("cuda")
	caption_1 = BlipCaption("cuda")
else:
	caption_0 = BlipCaption("cpu")
	caption_1 = BlipCaption("cpu")

In [ ]:
%%time

filtered['caption'] = filtered.progress_apply(lambda x: functions.apply_caption(x, [caption_0, caption_1]), axis=1)

display("== Filtered Data With Caption ==")
display(filtered.shape)
display(filtered)

In [ ]:
%%time

filtered_more = filtered.loc[(filtered['caption'] != "") & (filtered['caption'].notnull()) & (filtered['exists'] == True)]

display("== Filtered On Missing Caption ==")
display(filtered_more.shape)
display(filtered_more)

In [ ]:
%%time

dropped = filtered_more.dropna(axis=1, how='all')
dropped.reset_index(drop=True, inplace=True)

display("== Dropped And Reset Data ==")
display(dropped.shape)
display(dropped)

In [ ]:
%%time

current = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system)

display(current.shape)
display(current)

In [ ]:
%%time

temp_current = current
temp_current.set_index('id', inplace=True, drop=False)

temp_new = dropped
temp_new.set_index('id', inplace=True, drop=False)

temp_new.loc[~temp_new.index.isin(temp_current.index)]

In [ ]:
%%time

concat = pd.concat([current, dropped])

display("== Concatenated Data ==")
display(concat.shape)
display(concat)

In [ ]:
%%time

file_list_ = file_system.ls("data/image")
concat['path'] = concat.progress_apply(lambda x: functions.fix_path(x, file_list), axis=1)

display("== Concatenated Data With Path ==")
display(concat.shape)
display(concat)

In [ ]:
%%time

back_up_name = f"data/parquet/back_{dt.datetime.timestamp(dt.datetime.now())}.parquet"
display(f"== Writing Back-Up {back_up_name} ==")
current.to_parquet(back_up_name, schema=schema, filesystem=file_system)

In [ ]:
%%time

concat.to_parquet("data/parquet/back.parquet", schema=schema, filesystem=file_system)
new = pd.read_parquet("data/parquet/back.parquet", engine='pyarrow', schema=schema, filesystem=file_system)

display("== New Data ==")
display(new.shape)
display(new)

In [ ]:
!jupyter notebook stop